In [176]:
import numpy as np
import pandas as pd
from tensorflow import keras as tfk

In [177]:
# Get limit order book data 
orderbook = pd.read_csv('/Users/tanvipotdar/Projects/LOBster/data_tqap/INTC_2015-01-01_2015-01-31_10/INTC_2015-01-02_34200000_57600000_orderbook_10.csv')

In [178]:
# Normalise the data
from scipy.stats import zscore
normalised_data = orderbook.apply(zscore)
normalised_data.head(10)

,ask_price_1,ask_size_1,bid_price_1,bid_size_1,ask_price_2,ask_size_2,bid_price_2,bid_size_2,ask_price_3,ask_size_3,...,bid_price_8,bid_size_8,ask_price_9,ask_size_9,bid_price_9,bid_size_9,ask_price_10,ask_size_10,bid_price_10,bid_size_10
0,0.796834,-0.070811,0.114857,-1.063739,0.831288,-1.895349,0.011343,-1.984987,0.934676,-1.788479,...,-0.816904,-0.872724,2.450812,-0.888307,-0.885917,-1.164202,2.58852,-0.313841,-1.092996,-1.491542
1,0.796834,-0.070811,0.114857,-1.063739,0.831288,-1.895349,0.011343,-1.984987,0.934676,-1.788479,...,-0.816904,-0.872724,2.450812,-0.888307,-0.885917,-1.164202,2.58852,-0.313841,-1.092996,-1.491542
2,0.796834,-0.070811,0.114857,-1.154853,0.831288,-1.895349,0.011343,-1.984987,0.934676,-1.788479,...,-0.816904,-0.872724,2.450812,-0.888307,-0.885917,-1.164202,2.58852,-0.313841,-1.092996,-1.491542
3,0.796834,-0.070811,0.114857,-1.154853,0.831288,-1.895349,0.011343,-1.984987,0.934676,-1.788479,...,-0.437242,-0.872724,2.450812,-0.888307,-0.782372,-1.164202,2.58852,-0.313841,-0.851389,-1.491542
4,0.796834,-0.070811,0.114857,-1.154853,0.831288,-1.895349,0.011343,-1.984987,0.934676,-1.788479,...,-0.437242,-0.872724,2.450812,-0.888307,-0.782372,-1.164202,2.58852,-0.313841,-0.851389,-1.491542
5,0.796834,-0.070811,0.114857,-1.154853,0.831288,-1.895349,0.011343,-1.984987,0.934676,-1.788479,...,-0.437242,-0.872724,2.450812,-0.888307,-0.782372,-1.164202,2.58852,-0.313841,-0.851389,-1.491542
6,0.796834,-0.070811,0.114857,-1.154853,0.831288,-1.895349,0.011343,-1.984987,0.934676,-1.788479,...,-0.437242,-0.872724,2.450812,-0.888307,-0.782372,-1.164202,2.58852,-0.313841,-0.851389,-1.491542
7,0.796834,-0.070811,0.114857,-1.154853,0.831288,-1.895349,0.011343,-1.984987,0.934676,-1.788479,...,-0.437242,-0.872724,2.450812,-0.888307,-0.782372,-1.164202,2.58852,-0.313841,-0.851389,-1.491542
8,0.796834,-0.070811,0.632554,-1.154853,0.831288,-1.895349,0.149397,-2.053207,0.934676,-1.788479,...,-0.402727,-0.894391,2.450812,-0.888307,-0.402710,-1.164202,2.58852,-0.313841,-0.747843,-1.491542
9,0.796834,-0.070811,0.632554,-1.124482,0.831288,-1.895349,0.149397,-2.053207,0.934676,-1.788479,...,-0.402727,-0.894391,2.450812,-0.888307,-0.402710,-1.164202,2.58852,-0.313841,-0.747843,-1.491542


In [179]:
# calculate the midprice
normalised_data['midprice'] = (normalised_data.ask_price_1+normalised_data.bid_price_1)/2

In [180]:
# smoothed labelling of the midprice/ k is the prediction horizon
k = 10
# mean of previous k mid-prices
normalised_data['m_minus'] = normalised_data['midprice'].rolling(window=k).mean()
# mean of next k mid-prices
normalised_data['m_plus'] = normalised_data['midprice'][::-1].rolling(window=k).mean()[::-1]

In [202]:
# label the smoothed mid-prices based on a threshold/ alpha is the threshold 
alpha = 0.0001
normalised_data['change'] = (normalised_data.m_plus - normalised_data.m_minus)/normalised_data.m_minus
# assign categories up, down, stationary
normalised_data['label'] = pd.cut(normalised_data.change, bins=[-np.inf, -alpha, alpha, np.inf], 
                                  labels=['down', 'stationary', 'up'])
# drop all unlabelled values (will be first and last k values as they have no m_minus/m_plus value)
normalised_data.dropna(inplace=True)
normalised_data.head()

,ask_price_1,ask_size_1,bid_price_1,bid_size_1,ask_price_2,ask_size_2,bid_price_2,bid_size_2,ask_price_3,ask_size_3,...,bid_size_9,ask_price_10,ask_size_10,bid_price_10,bid_size_10,midprice,m_minus,m_plus,change,label
9,0.796834,-0.070811,0.632554,-1.124482,0.831288,-1.895349,0.149397,-2.053207,0.934676,-1.788479,...,-1.164202,2.58852,-0.313841,-0.747843,-1.491542,0.714694,0.507615,0.714694,0.407945,up
10,0.796834,-0.070811,0.632554,-1.124482,0.831288,-1.895349,0.149397,-2.053207,0.934676,-1.788479,...,-1.164202,2.58852,-0.313841,-0.747843,-1.491542,0.714694,0.533500,0.714694,0.339633,up
11,0.796834,-0.070811,0.632554,-1.124482,0.831288,-1.895349,0.149397,-2.053207,0.934676,-1.788479,...,-1.164202,2.58852,-0.313841,-0.747843,-1.491542,0.714694,0.559385,0.714694,0.277643,up
12,0.796834,-0.070811,0.632554,-1.094110,0.831288,-1.895349,0.149397,-2.053207,0.934676,-1.788479,...,-1.164202,2.58852,-0.313841,-0.747843,-1.491542,0.714694,0.585270,0.714694,0.221136,up
13,0.796834,-0.070811,0.632554,-1.094110,0.831288,-1.895349,0.149397,-2.053207,0.934676,-1.788479,...,-1.197339,2.58852,-0.313841,-0.368176,-1.491542,0.714694,0.611155,0.714694,0.169416,up


In [216]:
# get input and output train and test data
data = normalised_data[:887600]

cols = data.columns.to_list()[:40]
input_data = data[cols]
input_array = input_data.to_numpy().reshape(8876,100,40,1)

from sklearn.preprocessing import OneHotEncoder
output_data = data.label.to_numpy()[::-100][::-1]
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = output_data.reshape(len(output_data), 1)
output_array = onehot_encoder.fit_transform(integer_encoded)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(input_array, output_array, shuffle=False)

array([[[[ 0.79683389],
         [-0.070811  ],
         [ 0.63255445],
         ...,
         [-0.31384099],
         [-0.74784332],
         [-1.49154223]],

        [[ 0.79683389],
         [-0.070811  ],
         [ 0.63255445],
         ...,
         [-0.31384099],
         [-0.74784332],
         [-1.49154223]],

        [[ 0.79683389],
         [-0.070811  ],
         [ 0.63255445],
         ...,
         [-0.31384099],
         [-0.74784332],
         [-1.49154223]],

        ...,

        [[ 0.69343383],
         [-1.27175119],
         [ 0.11485709],
         ...,
         [-0.91119984],
         [-0.16108448],
         [-1.38236296]],

        [[ 0.69343383],
         [-1.27175119],
         [ 0.11485709],
         ...,
         [-0.91119984],
         [-0.16108448],
         [-1.38236296]],

        [[ 0.69343383],
         [-1.27175119],
         [ 0.11485709],
         ...,
         [-0.91119984],
         [-0.16108448],
         [-1.38236296]]],


       [[[ 0.69343383],


In [213]:
# convolutional layers
model = tfk.Sequential()
model.add(tfk.layers.Conv2D(filters=16, kernel_size=(1,2), input_shape=(100,40,1), strides=(1, 2)))
model.add(tfk.layers.LeakyReLU(alpha=0.01))
model.add(tfk.layers.Conv2D(filters=16, kernel_size=(1,2), strides=(1, 2)))
model.add(tfk.layers.LeakyReLU(alpha=0.01))
model.add(tfk.layers.Conv2D(filters=16, kernel_size=(1,10), input_shape=(100,10,1)))
model.add(tfk.layers.TimeDistributed(tfk.layers.Flatten()))

# lstm layer
model.add(tfk.layers.LSTM(64))
model.add(tfk.layers.Dense(3,activation='softmax'))

# compile model and summarize
adam = tfk.optimizers.Adam(lr=0.01, epsilon=1)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_355 (Conv2D)          (None, 100, 20, 16)       48        
_________________________________________________________________
leaky_re_lu_143 (LeakyReLU)  (None, 100, 20, 16)       0         
_________________________________________________________________
conv2d_356 (Conv2D)          (None, 100, 10, 16)       528       
_________________________________________________________________
leaky_re_lu_144 (LeakyReLU)  (None, 100, 10, 16)       0         
_________________________________________________________________
conv2d_357 (Conv2D)          (None, 100, 1, 16)        2576      
_________________________________________________________________
time_distributed_35 (TimeDis (None, 100, 16)           0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 64)                20736     
__________

In [ ]:
BATCH_SIZE = 100
EPOCHS = 100
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)


Epoch 1/100
6657/6657 [==============================] - 8s 1ms/sample - loss: 0.6736 - acc: 0.8583
Epoch 2/100
6657/6657 [==============================] - 8s 1ms/sample - loss: 0.5055 - acc: 0.8659
Epoch 3/100
6657/6657 [==============================] - 8s 1ms/sample - loss: 0.4956 - acc: 0.8659
Epoch 4/100
6657/6657 [==============================] - 7s 1ms/sample - loss: 0.4920 - acc: 0.8659
Epoch 5/100
6657/6657 [==============================] - 8s 1ms/sample - loss: 0.4897 - acc: 0.8659
Epoch 6/100
6657/6657 [==============================] - 8s 1ms/sample - loss: 0.4880 - acc: 0.8659
Epoch 7/100
6657/6657 [==============================] - 8s 1ms/sample - loss: 0.4868 - acc: 0.8659
Epoch 8/100
6657/6657 [==============================] - 8s 1ms/sample - loss: 0.4859 - acc: 0.8659
Epoch 9/100
6657/6657 [==============================] - 7s 1ms/sample - loss: 0.4851 - acc: 0.8659
Epoch 10/100
6657/6657 [==============================] - 7s 1ms/sample - loss: 0.4843 - acc: 0.8659